In [ ]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from matplotlib import patches as ptc 
import math
import random
from tensorflow.keras.utils import plot_model

from PIL import Image
import os



In [ ]:
(x_train0, y_train0), (x_test0, y_test0) = tf.keras.datasets.mnist.load_data()

x_train = np.asarray(x_train0, dtype=np.float32).reshape(60000, 28*28) / 255.0
x_test = np.asarray(x_test0, dtype=np.float32).reshape(10000, 28*28) / 255.0
y_train = np.asarray(y_train0, dtype=np.int32)
y_test = np.asarray(y_test0, dtype=np.int32)

y_train_onehot =  tf.one_hot( y_train, depth=10, dtype=tf.float32)
y_test_onehot =  tf.one_hot( y_test, depth=10, dtype=tf.float32)   


In [ ]:
num_img = 5
plt.imshow(x_train0[num_img], cmap='gray')
plt.title(f"Label: {y_train0[num_img]}")


In [ ]:
print(x_train.shape)
print(y_train_onehot.shape)


# Why we use TensorFlow?
* Our code will run on GPUs (Not today's tutorial)! Much faster training. 
* We want you to be ready to use the framework for your project so you can experiment more efficiently than with the code we wrote by hand. 
* We want you to stand on the shoulders of giants! TensorFlow and PyTorch are both excellent frameworks that will make your lives a lot easier :)
 

# Keras Sequential API
In Keras, there's easy way to define the neural network which is expressed as a sequential stack of layers, with the outpupt of each layer fed to the next layer as input. Simply call the `tf.keras.Sequential` constructor with a list containing a sequence of layer objects.


To define each layers, we will pick one type of layer from `tf.keras.layers`. 



In [ ]:
def two_layer_net():
    model = tf.keras.Sequential(layers = [
        tf.keras.layers.InputLayer(input_shape=(28,28,1),name = 'input'),
        tf.keras.layers.Conv2D(5, 3,activation= 'relu',name = 'conv'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation='softmax',name = 'output')
    ])

    return model

In [ ]:
model = two_layer_net()
model.summary()
#tf.keras.utils.plot_model(model, "model.png")
#plot_model(model, to_file = 'neuralnet_model.png', show_shapes=True, show_layer_names=True)

To train the network, we defined the loss function (ex.MSE or CrossEntropy) and minimize the loss function using the Stochastic Gradient Decent, which requires to compute the gradient of loss function with respect to each weights and bias. The way of computing each gradient is called back-progagation.  In Keras seqeuntial API, `.complie()` deals with every jobs above. 


In [ ]:
def two_layer_net():
    model = tf.keras.Sequential(layers = [
        tf.keras.layers.InputLayer(input_shape=(28,28,1),name = 'input'),
        tf.keras.layers.Conv2D(5, 3, activation= 'relu',name = 'conv'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation='softmax',name = 'output')
              

    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model



In [ ]:
x_train0.shape

Once we decide the complier, we need to train the model by fitting on the training data. 

In [ ]:
model = two_layer_net()
training_history = model.fit(x_train0.reshape(60000,28,28,1), y_train_onehot, epochs=10)

In [ ]:
Loss = training_history.history['loss']
Accuracy = training_history.history['accuracy']
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(Loss, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
              

plt.subplot(1, 2, 2)
plt.plot(Accuracy, label='Training Accuracy', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training Accuracy over Epochs')

plt.show()      

In [ ]:
model.evaluate(x_test.reshape(10000,28,28,1), y_test_onehot)

In [ ]:
plt.imshow(x_train0[5], cmap='gray')
pred = model.predict(x_train[5:6].reshape(1,28,28))

In [ ]:
pred

In [ ]:
for i, img in enumerate(x_test0[0:64]):
    plt.subplot(8, 8, i+1)
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    #prediction = model.predict(x_test[i:i+1])
    #pred_label = np.argmax(prediction)
    
    #plt.title(f' {pred_label,np.round(prediction[0,pred_label],decimals=3)}')
    plt.tight_layout()
        

In [ ]:
filters = model.layers[0].get_weights()[0]
np.shape(filters)

In [ ]:
m = np.abs(filters).max()
plt.imshow(filters[:,:,:,0].reshape(3,3), cmap='gray', vmin=-m, vmax=m)

In [ ]:
cols = 10
rows = 10
fig, axes = plt.subplots(rows, cols, figsize=(20,20))
idxs = range(rows*cols)
m = np.abs(weights).max()
for ax, j in zip(axes.ravel(), idxs):
    ax.imshow(weights[:, j].reshape(28,28), cmap='gray', vmin=-m, vmax=m)
    ax.set_title(str(j), fontsize=8)
    ax.axis('off')
plt.suptitle("hidden1 weights")
plt.tight_layout()
plt.show()

## Convolutional Neural Network

In [ ]:

def convolution(input, filter):
    r_filter, c_filter = filter.shape
    row = input.shape[0] - r_filter + 1
    col = input.shape[1] - c_filter + 1
   
    output = np.zeros((row, col))

    for i in range(row):
        for j in range(col):
            part = input[i:i+r_filter,j:j+c_filter]
            output[i,j] = (filter* part).sum()

    return output



In [ ]:
Image = np.array([[0, 0, 1, 0],[0, 1,0.5,0], [1,0,1,0],[1,0.5,1,1]])
Filter = np.array([[1,0,-1],[1,0,-1],[1,0,-1]])
Output = convolution(Image, Filter)
Filter

In [ ]:

num_img0 =1
input0 = x_train0[num_img0]
plt.imshow(input, cmap='gray')

plt.title('size of image is %d x %d '%(input0.shape[0], input0.shape[1]),fontsize = 20)
plt.show()

In [ ]:
output = convolution(input0, Filter)
plt.imshow(output, cmap='gray')
#plt.title('what is the size of output?',fontsize = 20)
plt.title('Size of image is %d x %d '%(output.shape[0], output.shape[1]),fontsize = 20)

In [ ]:
filters = [np.array([[0,0,0],[0,1,0],[0,0,0]]),np.array([[1,0,-1],[1,0,-1],[1,0,-1]]), np.array([[1,1,1],[0,0,0],[-1,-1,-1]]), np.array([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]]),np.array([[1,1,1],[1,1,1],[1,1,1]])] #np.array([[0,-1,0],[-1,5,-1],[0,-1,0]])]  
name_filters = ['identity', 'vertical edge detector', 'horizontal edge detector', 'edge detector', 'blur']
cols = 4
rows = 5

num_img1 =6
input1 = x_train0[num_img1]

num_img5 =239
input5 = x_train0[num_img5]

fig, axes = plt.subplots(rows, cols, figsize=(20, 20))

for i in range(rows):
    filter = filters[i] 
    axes[ i,0].imshow(filters[i], cmap='gray')
    axes[ i,0].set_title(name_filters[i])
    axes[ i,0].axis('off')

    
    for x in range(filter.shape[1] + 1):
        axes[i,0].axvline(x - 0.5, color='yellow', linewidth=1)
    for y in range(filter.shape[0] + 1):
        axes[i,0].axhline(y - 0.5, color='yellow', linewidth=1)
        
    
    for y in range(filter.shape[0]):
        for x in range(filter.shape[1]):
            axes[i,0].text(x, y, f"{filter[y, x]:.0f}",
                    ha='center', va='center', color='blue',
                    fontsize=14, fontweight='bold')

    # Right: output
    axes[i,1].imshow(convolution(input0,filter), cmap='gray')
    axes[i,1].axis('off')
    axes[i,2].imshow(convolution(input1,filter), cmap='gray')
    axes[i,2].axis('off')
    axes[i,3].imshow(convolution(input5,filter), cmap='gray')
   
    
plt.tight_layout()

In [ ]:
filters = [np.array([[0,0,0],[0,1,0],[0,0,0]]),np.array([[1,0,-1],[1,0,-1],[1,0,-1]]), np.array([[1,1,1],[0,0,0],[-1,-1,-1]]), np.array([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]]),np.array([[1,1,1],[1,1,1],[1,1,1]])] #np.array([[0,-1,0],[-1,5,-1],[0,-1,0]])]  
name_filters = ['identity', 'vertical edge detector', 'horizontal edge detector', 'edge detector', 'blur']
cols = 5
rows = 2
fig, axes = plt.subplots(rows, cols, figsize=(20, 10))

for i in range(cols):
    filter = filters[i] 
    axes[ 0,i].imshow(filters[i], cmap='gray')
    axes[ 0,i].set_title(name_filters[i])
    axes[ 0,i].axis('off')

    
    for x in range(filter.shape[1] + 1):
        axes[0,i].axvline(x - 0.5, color='yellow', linewidth=1)
    for y in range(filter.shape[0] + 1):
        axes[0,i].axhline(y - 0.5, color='yellow', linewidth=1)
        
    
    for y in range(filter.shape[0]):
        for x in range(filter.shape[1]):
            axes[0,i].text(x, y, f"{filter[y, x]:.0f}",
                    ha='center', va='center', color='blue',
                    fontsize=14, fontweight='bold')

    # Right: output
    axes[1,i].imshow(convolution(input,filter), cmap='gray')
    axes[1,i].axis('off')
    
plt.tight_layout()

In [ ]:
np.where(y_train == 1)[0][:10]

In [ ]:

num_img0 =239
input0 = x_train0[num_img0]
plt.imshow(input0, cmap='gray')

In [ ]:
np.where(y_train == 5)[0][:20]

# Tensorflow for CNN

In [ ]:
(x_train0, y_train0), (x_test0, y_test0) = tf.keras.datasets.mnist.load_data()

x_train = x_train0.reshape(60000, 28, 28, 1).astype("float32") / 255.0
x_test = x_test0.reshape(10000, 28, 28, 1).astype("float32") / 255.0
y_train = np.asarray(y_train0, dtype=np.int32)
y_test = np.asarray(y_test0, dtype=np.int32)

y_train_onehot =  tf.one_hot( y_train, depth=10, dtype=tf.float32)
y_test_onehot =  tf.one_hot( y_test, depth=10, dtype=tf.float32)

In [ ]:
def two_layer_cov():
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(shape=(28, 28, 1), name='input'),
        tf.keras.layers.Conv2D(5, 7, activation='relu', strides = 2, name='conv1'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation='softmax', name='output')
    ])
    return model

In [ ]:
model2 = two_layer_cov()
plot_model(model2, to_file = 'neuralnet_model.png', show_shapes=True, show_layer_names=True)

In [ ]:
model2 = two_layer_cov()
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


training_history2 = model2.fit(x_train, y_train, epochs=5)


In [ ]:
test_loss, test_acc = model2.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")

In [ ]:
Loss = training_history2.history['loss']
Accuracy = training_history2.history['accuracy']
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(Loss, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
              

plt.subplot(1, 2, 2)
plt.plot(Accuracy, label='Training Accuracy', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training Accuracy over Epochs')

plt.show()    

In [ ]:
filters = model2.layers[0].get_weights()[0]

#[0].get_weights()[0]
#np.shape(filters)

In [ ]:
cols = 5
rows = 1
fig, axes = plt.subplots(rows, cols, figsize=(20,4))
idxs = range(rows*cols)
m = np.abs(filters).max()
for ax, j in zip(axes.ravel(), idxs):
    ax.imshow(filters[:,:,:, j].reshape(7,7), cmap='gray', vmin=-m, vmax=m)
    ax.set_title(str(j), fontsize=8)
    ax.axis('off')
plt.suptitle("hidden1 weights")
plt.tight_layout()
plt.show()

In [ ]:
def three_layer_cov():
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(shape=(28, 28, 1), name='input'),
        tf.keras.layers.Conv2D(5, 7, activation='relu', name='conv1'),
        tf.keras.layers.Conv2D(10, 3, activation='relu', name='conv2'), 
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation='softmax', name='output')
    ])
    return model

In [ ]:
model3 = three_layer_cov()
plot_model(model3, to_file = 'neuralnet_model.png', show_shapes=True, show_layer_names=True)
model3.summary()

In [ ]:
def three_layer_pooling_cov():
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(shape=(28, 28, 1), name='input'),
        tf.keras.layers.Conv2D(5, 7, activation='relu', name='conv1'),
        tf.keras.layers.MaxPooling2D(2, name='pool1'),
        tf.keras.layers.Conv2D(10, 3, activation='relu', name='conv2'),
        tf.keras.layers.MaxPooling2D(2, name='pool2'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation='softmax', name='output')
    ])
    return model

In [ ]:
model4 = three_layer_pooling_cov()
plot_model(model4, to_file = 'neuralnet_model.png', show_shapes=True, show_layer_names=True)
model4.summary()

In [ ]:
model4= three_layer_pooling_cov()
model4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


training_history4 = model4.fit(x_train, y_train, epochs=5)
test_loss, test_acc = model4.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")

In [ ]:
(x_train0, y_train0), (x_test0, y_test0) = tf.keras.datasets.cifar10.load_data()
x_train = x_train0.reshape(50000, 32, 32, 3).astype("float32") / 255.0
x_test = x_test0.reshape(10000, 32, 32, 3).astype("float32") / 255.0
y_train = np.asarray(y_train0, dtype=np.int32)
y_test = np.asarray(y_test0, dtype=np.int32)

In [ ]:
for i, img in enumerate(x_test0[0:64]):
    plt.subplot(8, 8, i+1)
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    #prediction = model.predict(x_test[i:i+1])
    #pred_label = np.argmax(prediction)
    
   
    plt.tight_layout()
        

In [ ]:
def NerualNet_3_channel():

    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(shape=(32, 32, 3), name='input'),
        tf.keras.layers.Conv2D(32, 3, activation='relu', name='conv1'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.MaxPooling2D(2, name='pool1'),
        tf.keras.layers.Conv2D(10, 3, activation='relu', name='conv2'),
        tf.keras.layers.MaxPooling2D(2, name='pool2'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation='softmax', name='output')
       
    ])
    return model

  


In [ ]:
model5 = NerualNet_3_channel()
model5.summary()

In [ ]:
model5 = NerualNet_3_channel()
model5.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


training_history5 = model5.fit(x_train, y_train, epochs=30)